# Training A ML Models From Scratch

> A demonstration of ML pipelines and how to develop them


## About

This notebook is to help those attending my 2024 Intel oneAPI Workshop on reusing and extending deep learning models.

This notebook, while it does not provide any deep learning code, does lay the foundational information for:

1. What is a machine learning pipeline,
2. How to train a machine learning model,
3. How to evaluate a machine learning model,
4. How to execute a machine learning model on new data

## Pipelines

A *data pipeline* can be thought of as a series of steps or actions to manipulate data to adhere to a specified format **without**.

A common data pipeline pattern is *ETL*, or *Extract, Transform, Load*.

*Machine learning pipelines* expand on ETL by including further steps to train, evaluate, refine, and deploy machine learning models.

### Data Pipeline

From IBM:

```
A data pipeline is a method in which raw data is ingested from various data sources, transformed and then ported to a data store, such as a data lake or data warehouse, for analysis.
```

- [IBM, *What is a data pipeline?*](https://www.ibm.com/topics/data-pipeline)

To illistrate the flow of data, see the below figure:

![Data pipeline example image](images/data_pipeline_example.png)

**NOTE:** I intentionally did not add any detail on *how* data flows through the pipeline. Pipelines, at a high-level, can be thought of implementation agnostic details of a larger application. 